# Python: Identify areas of potential foliage influence on railway network

In [ ]:
import geopandas as gpd
import requests
import shapely
import folium

## Use GeoPandas to load geodata and to clip it to an area of interest

Add Deutsche Bahn Streckennetz from Web Feature Service ([WFS](https://python-gis-book.readthedocs.io/en/latest/part2/chapter-09/nb/01-retrieving-data-from-wfs.html))

In [ ]:
# Specify the base WFS url of the service
railway_wfs_url = 'https://geoserver.geonet-mrn.de/xdatatogo/db_strecken/ows'

# Specify HTTP GET parameters
railway_wfs_params = dict(
    service='WFS',
    request='GetFeature',
    typeNames='xdatatogo:db_strecken',
    outputFormat='json',
)

# Build the request by adding all parameters to the service URL
request = requests.Request('GET', railway_wfs_url, params=railway_wfs_params).prepare()

How does such an HTTP request look like?

In [ ]:
request.url

Now retrieve data from server into new GeoDataFrame

In [ ]:
railway_network = gpd.read_file(request.url)

Visualise downloaded data in map

In [ ]:
map = railway_network.explore(color='black')
map

Clip the railway network to WGS 84 ([EPSG:4326](https://epsg.io/4326)) bounding box of Lower Saxony. The BBOX is defined by its corner points: `(min_long, min_lat), (max_long, max_lat)`.

In [ ]:
bbox = shapely.envelope(
    shapely.MultiPoint([(6.345854, 51.295232), (11.598078, 54.13791)])
)

railway_network = railway_network.clip(bbox)

In [ ]:
map = railway_network.explore(name='clipped railway network', color='black')
folium.GeoJson(bbox, name='bounding box', color='grey').add_to(map)
folium.LayerControl().add_to(map)
map

## Project the extracted layer to Cartesian 2D coordinates in order to be able to use 'metre' unit for measuring and buffering

As destination projection we use the governmentally recommended UTM32 ([EPSG:25832](https://epsg.io/25832)) projection

In [ ]:
print('Previous CRS:', railway_network.crs)
railway_network = railway_network.to_crs(epsg=25832)
print('New CRS:', railway_network.crs)

## Create a buffer around the railway tracks

In [ ]:
buffer_distance_m = 20

# Duplicate layer to separately store it with a buffered geometry
railway_network_buffer = railway_network.copy()
railway_network_buffer['geometry'] = railway_network['geometry'].buffer(buffer_distance_m)

Inspect the data table if it contains `Polygon` geometries now instead of `(Multi)LineString`.

In [ ]:
railway_network_buffer.head()

## Identifying areas of foliage influence on railway network

Load [forest dataset from WFS](https://mis.bkg.bund.de/trefferanzeige?docuuid=75C069E4-D760-49FF-BD71-5188CF81B4D9) of BKG.

- The relevant dataset "AX_Wald" is identified by the cryptic layer name `dlm250:objart_43002_f`.
- This governmental server returns geometries referenced by default in UTM32 (EPSG:25832). Thus, we can also skip additional re-projection.
- We also skip a subsequent clipping step by directly including the BBOX as spatial filter in the WFS request. Because our BBOX is specified in WGS 84 longitude/latitude, we have to explicitly provide the reference system identifier EPSG:4326 for the server to correctly interpret the BBOX coordinates. 

In [ ]:
# Add reference system identifier to bbox parameter
bbox_parameter = ','.join(str(coord) for coord in bbox.bounds) + ',EPSG:4326'

forest_wfs_url = 'https://sgx.geodatenzentrum.de/wfs_dlm250'

forest_wfs_params = dict(
    service='WFS',
    request='GetFeature',
    typeNames='dlm250:objart_43002_f',
    outputFormat='json',
    bbox=bbox_parameter
)

request = requests.Request('GET', forest_wfs_url, params=forest_wfs_params).prepare()

forest_patches = gpd.read_file(request.url)

Perform spatial [intersection](https://en.wikipedia.org/wiki/Vector_overlay#Operators) of our railway network with the forest patches

In [ ]:
result = gpd.overlay(railway_network_buffer, forest_patches, how='intersection')

## Plot the result

In [ ]:
map = railway_network.explore(name='clipped railway network', color='black')
folium.GeoJson(forest_patches['geometry'], name='forest', color='darkgreen', stroke=False, show=False).add_to(map)
folium.GeoJson(result['geometry'], name='susceptible areas', color='red').add_to(map)
folium.LayerControl().add_to(map)
map

## Optional: save map as HTML website and export result to a file

In [ ]:
map.save('susceptible_railway_areas.html')

output_file = 'railway_network_intersect_forest.json'
result.to_file(output_file)
print('File exported successfully.')